In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
from tensorflow.keras.metrics import AUC
from tensorflow.nn import softmax, sigmoid
from PIL import Image
import os

In [2]:
#Load Test data 
def loadData(m,path):
    dirlist=[(i,os.path.join(path,i)) for i in os.listdir(path)]
    x = np.zeros(shape=(m,224,224,3))
    y = np.zeros(shape=(m,1),dtype=int)

    y_cat=np.zeros(shape=(m,1),dtype=int)
   
    
    shift=0
    for idx,path in dirlist:
        imgList=[os.path.join(path,k) for k in os.listdir(path)]
        
        for p in range(len(imgList)):
            x[p+shift]=np.array(Image.open(imgList[p]).convert("RGB"))      
            y[p+shift]=(idx!='0')
            y_cat[p+shift]=int(idx)
        shift+=len(imgList)
    
    
    return  x,y,y_cat
                      

In [ ]:
#Load test data
test_Path="Enter test data folder path here"
x_test,y_test,y_testSev=loadData(1656,test_Path)
preprocess_input(x_test)

In [4]:
# Severity Grading (Categorical Model) Architecture
vgg=VGG16(input_shape=(224,224,3),weights="imagenet",include_top=False)
final_output=Flatten()(vgg.output)
final_output=Dense(units=512,activation ='leaky_relu')(final_output)
final_output=Dense(units=512,activation ='leaky_relu')(final_output)
final_output=Dense(units=5,activation="linear")(final_output)
model=Model(inputs=vgg.input,outputs=final_output)

In [5]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model.load_weights("Enter path for trained weights")

In [7]:
model.evaluate(x_test,y_testSev,batch_size=32)

52/52 [==============================] - 20s 224ms/step - loss: 0.7363 - sparse_categorical_accuracy: 0.6975


[0.7363157272338867, 0.6974637508392334]

In [4]:
# Osteoarthritis Detection (Binary Model) Architecture
vgg_p=VGG16(input_shape=(224,224,3),weights="imagenet",include_top=False)
final_output_p=Flatten()(vgg_p.output)
final_output_p=Dense(units=512,activation ='leaky_relu')(final_output_p)
final_output_p=Dense(units=512,activation ='leaky_relu')(final_output_p)
final_output_p=Dense(units=1,activation="linear")(final_output_p)
model_p=Model(inputs=vgg_p.input,outputs=final_output_p)

In [6]:
model_p.compile(loss=BinaryCrossentropy(from_logits=True),metrics=['accuracy',AUC(from_logits=True)])
model_p.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
model_p.load_weights("Enter path for trained weights")

In [8]:
model_p.evaluate(x_test,y_test,batch_size=32)

52/52 [==============================] - 19s 223ms/step - loss: 0.3683 - accuracy: 0.8357 - auc: 0.9121


[0.36826634407043457, 0.8357487916946411, 0.9121353030204773]

In [ ]:
#Accuracy measurement using both models together (single batch)
acc=np.zeros(#enter number of testing example (int),dtype=int)
accs=np.zeros(#enter number of testing example (int),dtype=int)
for case in range(len(x_test)):
    tmp=x_test[case].reshape(-1,224,224,3)
    if(sigmoid(model_p.predict(tmp))>0.5):
        
        acc[case]=int(y_test[case][0]==1)
        severity=np.argmax(softmax(model.predict(tmp)))
        accs[case]=int(y_testSev[case][0]==severity)
    else:
        acc[case]=int(y_test[case][0]==0)
        accs[case]=int(y_testSev[case][0]==0)
print(np.mean(acc))
print(np.mean(accs))